In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/yap15-heart-diseases-predictions/sample_submission.csv
/kaggle/input/yap15-heart-diseases-predictions/train.csv
/kaggle/input/yap15-heart-diseases-predictions/test.csv


In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import pandas_profiling

import warnings
warnings.filterwarnings('ignore')


from sklearn.model_selection import train_test_split, GridSearchCV, RandomizedSearchCV
from sklearn.model_selection import cross_val_score

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder, OneHotEncoder
from sklearn.metrics import f1_score, roc_auc_score, classification_report

from sklearn.feature_selection import SelectKBest,chi2,RFE

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression

# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)
 
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
 
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

In [3]:
df_train = pd.read_csv('/kaggle/input/yap15-heart-diseases-predictions/train.csv',index_col=0)
df_test = pd.read_csv('/kaggle/input/yap15-heart-diseases-predictions/test.csv',index_col=0)
display(df_train.head())
display(df_test.head())
display(df_train.columns)
display(df_train.info())
print('Размер  тренировочной таблицы:', df_train.shape)
print('Размер  тестовой таблицы:', df_test.shape)

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
id,,,,,,,,,,,
5,18888,1,154,85.0,130,80,1,1,0,0,1
6,19042,2,170,69.0,130,90,1,1,0,0,1
7,20432,1,160,70.0,120,75,1,1,0,0,0
10,18133,2,185,94.0,130,90,1,1,0,0,1
11,16093,2,170,76.0,120,80,1,1,0,0,1


Index(['age', 'gender', 'height', 'weight', 'ap_hi', 'ap_lo', 'cholesterol',
       'gluc', 'smoke', 'alco', 'active', 'cardio'],
      dtype='object')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 70000 entries, 0 to 99999
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          70000 non-null  int64  
 1   gender       70000 non-null  int64  
 2   height       70000 non-null  int64  
 3   weight       70000 non-null  float64
 4   ap_hi        70000 non-null  int64  
 5   ap_lo        70000 non-null  int64  
 6   cholesterol  70000 non-null  int64  
 7   gluc         70000 non-null  int64  
 8   smoke        70000 non-null  int64  
 9   alco         70000 non-null  int64  
 10  active       70000 non-null  int64  
 11  cardio       70000 non-null  int64  
dtypes: float64(1), int64(11)
memory usage: 6.9 MB


None

Размер  тренировочной таблицы: (70000, 12)
Размер  тестовой таблицы: (30000, 11)


In [4]:
df_train.isnull().sum()

age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
cardio         0
dtype: int64

In [5]:
df_test.isnull().sum()

age            0
gender         0
height         0
weight         0
ap_hi          0
ap_lo          0
cholesterol    0
gluc           0
smoke          0
alco           0
active         0
dtype: int64

In [6]:
# profile=pandas_profiling.ProfileReport(df_train)
# profile

In [7]:
df_train.corr()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
age,1.000000,-0.022811,-0.081515,0.053684,0.020764,0.017647,0.154424,0.098703,-0.047633,-0.029723,-0.009927,0.238159
gender,-0.022811,1.000000,0.499033,0.155406,0.006005,0.015254,-0.035821,-0.020491,0.338135,0.170966,0.005866,0.008109
height,-0.081515,0.499033,1.000000,0.290968,0.005488,0.006150,-0.050226,-0.018595,0.187989,0.094419,-0.006570,-0.010821
weight,0.053684,0.155406,0.290968,1.000000,0.030702,0.043710,0.141768,0.106857,0.067780,0.067113,-0.016867,0.181660
ap_hi,0.020764,0.006005,0.005488,0.030702,1.000000,0.016086,0.023778,0.011841,-0.000922,0.001408,-0.000033,0.054475
ap_lo,0.017647,0.015254,0.006150,0.043710,0.016086,1.000000,0.024019,0.010806,0.005186,0.010601,0.004780,0.065719
cholesterol,0.154424,-0.035821,-0.050226,0.141768,0.023778,0.024019,1.000000,0.451578,0.010354,0.035760,0.009911,0.221147
gluc,0.098703,-0.020491,-0.018595,0.106857,0.011841,0.010806,0.451578,1.000000,-0.004756,0.011246,-0.006770,0.089307
smoke,-0.047633,0.338135,0.187989,0.067780,-0.000922,0.005186,0.010354,-0.004756,1.000000,0.340094,0.025858,-0.015486
alco,-0.029723,0.170966,0.094419,0.067113,0.001408,0.010601,0.035760,0.011246,0.340094,1.000000,0.025476,-0.007330


In [8]:
df_train.duplicated().sum(), df_test.duplicated().sum() 

(24, 6)

In [9]:
df_train.drop_duplicates(keep='first', inplace=True)
#df_test.drop_duplicates(keep='first', inplace=True)    

In [10]:
df_train.duplicated().sum(), df_test.duplicated().sum() 

(0, 6)

In [11]:
df_train.head()

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,2,168,62.0,110,80,1,1,0,0,1,0
1,20228,1,156,85.0,140,90,3,1,0,0,1,1
2,18857,1,165,64.0,130,70,3,1,0,0,0,1
3,17623,2,169,82.0,150,100,1,1,0,0,1,1
4,17474,1,156,56.0,100,60,1,1,0,0,0,0


In [12]:
df_train['gender'] = df_train['gender'].map({1: 0, 2: 1})
df_test['gender'] = df_test['gender'].map({1: 0, 2: 1})

In [13]:
display(df_train.head())
display(df_test.head())

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,18393,1,168,62.0,110,80,1,1,0,0,1,0
1,20228,0,156,85.0,140,90,3,1,0,0,1,1
2,18857,0,165,64.0,130,70,3,1,0,0,0,1
3,17623,1,169,82.0,150,100,1,1,0,0,1,1
4,17474,0,156,56.0,100,60,1,1,0,0,0,0


,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
id,,,,,,,,,,,
5,18888,0,154,85.0,130,80,1,1,0,0,1
6,19042,1,170,69.0,130,90,1,1,0,0,1
7,20432,0,160,70.0,120,75,1,1,0,0,0
10,18133,1,185,94.0,130,90,1,1,0,0,1
11,16093,1,170,76.0,120,80,1,1,0,0,1


In [14]:
df_train['age'] = df_train['age'] / 365.25
df_train['age'] = df_train['age'].astype(int)

df_test['age'] = df_test['age'] / 365.25
df_test['age'] = df_test['age'].astype(int)

In [15]:
display(df_train.head())
display(df_test.head())

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
0,50,1,168,62.0,110,80,1,1,0,0,1,0
1,55,0,156,85.0,140,90,3,1,0,0,1,1
2,51,0,165,64.0,130,70,3,1,0,0,0,1
3,48,1,169,82.0,150,100,1,1,0,0,1,1
4,47,0,156,56.0,100,60,1,1,0,0,0,0


,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active
id,,,,,,,,,,,
5,51,0,154,85.0,130,80,1,1,0,0,1
6,52,1,170,69.0,130,90,1,1,0,0,1
7,55,0,160,70.0,120,75,1,1,0,0,0
10,49,1,185,94.0,130,90,1,1,0,0,1
11,44,1,170,76.0,120,80,1,1,0,0,1


In [16]:
df_train.describe().T

,count,mean,std,min,25%,50%,75%,max
age,69976.0,52.803475,6.762773,29.0,48.0,53.0,58.0,64.0
gender,69976.0,0.349648,0.476862,0.0,0.0,0.0,1.0,1.0
height,69976.0,164.359152,8.211218,55.0,159.0,165.0,170.0,250.0
weight,69976.0,74.208519,14.397211,10.0,65.0,72.0,82.0,200.0
ap_hi,69976.0,128.820453,154.037729,-150.0,120.0,120.0,140.0,16020.0
ap_lo,69976.0,96.636261,188.504581,-70.0,80.0,80.0,90.0,11000.0
cholesterol,69976.0,1.366997,0.680333,1.0,1.0,1.0,2.0,3.0
gluc,69976.0,1.226535,0.572353,1.0,1.0,1.0,1.0,3.0
smoke,69976.0,0.088159,0.283528,0.0,0.0,0.0,0.0,1.0
alco,69976.0,0.053790,0.225604,0.0,0.0,0.0,0.0,1.0


In [17]:
wrong_weight = df_train[df_train['weight'] <= 30]
display(wrong_weight.head())

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
5306,42,0,120,30.0,110,70,1,1,0,0,1,0
26503,49,0,160,30.0,120,80,1,1,0,0,1,1
38312,63,0,157,23.0,110,80,1,1,0,0,1,0
42156,55,1,177,22.0,120,80,1,1,1,1,1,0
48318,59,1,178,11.0,130,90,1,1,0,0,1,1


In [18]:
df_train.loc[wrong_weight.index, 'weight'] = None

df_train['weight'] = df_train['weight'].fillna(df_train['weight'].median())

display(df_train.loc[wrong_weight.index].head())

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
5306,42,0,120,72.0,110,70,1,1,0,0,1,0
26503,49,0,160,72.0,120,80,1,1,0,0,1,1
38312,63,0,157,72.0,110,80,1,1,0,0,1,0
42156,55,1,177,72.0,120,80,1,1,1,1,1,0
48318,59,1,178,72.0,130,90,1,1,0,0,1,1


In [19]:
big_height = df_train[df_train['height'] > 220]
display(big_height.head())
small_height = df_train[df_train['height'] < 100]
display(small_height.head())

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
9223,58,0,250,86.0,140,100,3,1,0,0,1,1


,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
309,59,1,76,55.0,120,80,1,1,0,0,1,0
10843,40,1,70,72.0,120,8,1,1,0,0,1,0
11662,48,1,97,170.0,160,100,1,1,1,0,1,1
18218,53,0,75,168.0,120,80,1,1,1,0,1,1
18928,61,1,71,68.0,120,80,3,1,0,0,1,0


In [20]:
df_train.loc[big_height.index, 'height'] = None
df_train.loc[small_height.index, 'height'] = None
df_train['height'] = df_train['height'].fillna(df_train['height'].median())

In [21]:
display(df_train.loc[big_height.index].head())
display(df_train.loc[small_height.index].head())

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
9223,58,0,165.0,86.0,140,100,3,1,0,0,1,1


,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
309,59,1,165.0,55.0,120,80,1,1,0,0,1,0
10843,40,1,165.0,72.0,120,8,1,1,0,0,1,0
11662,48,1,165.0,170.0,160,100,1,1,1,0,1,1
18218,53,0,165.0,168.0,120,80,1,1,1,0,1,1
18928,61,1,165.0,68.0,120,80,3,1,0,0,1,0


In [22]:
minus_ap_hi = df_train[df_train['ap_hi'] < 0]
display(minus_ap_hi.head())
minus_ap_lo = df_train[df_train['ap_lo'] < 0]
display(minus_ap_lo.head())

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
6525,41,0,165.0,78.0,-100,80,2,1,0,0,1,0
22881,60,1,161.0,90.0,-115,70,1,1,0,0,1,0
29313,42,0,153.0,54.0,-100,70,1,1,0,0,1,0
34295,50,0,162.0,74.0,-140,90,1,1,0,0,1,1
36025,40,1,168.0,50.0,-120,80,2,1,0,0,0,1


,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
85816,61,0,167.0,74.0,15,-70,1,1,0,0,1,1


In [23]:
df_train.loc[df_train['ap_hi'] < 0, 'ap_hi'] *= -1
display(df_train.loc[minus_ap_hi.index].head())
df_train.loc[df_train['ap_lo'] < 0, 'ap_lo'] *= -1
display(df_train.loc[minus_ap_lo.index].head())

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
6525,41,0,165.0,78.0,100,80,2,1,0,0,1,0
22881,60,1,161.0,90.0,115,70,1,1,0,0,1,0
29313,42,0,153.0,54.0,100,70,1,1,0,0,1,0
34295,50,0,162.0,74.0,140,90,1,1,0,0,1,1
36025,40,1,168.0,50.0,120,80,2,1,0,0,0,1


,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
85816,61,0,167.0,74.0,15,70,1,1,0,0,1,1


In [24]:
wrong_ap_hi = df_train[df_train['ap_hi'] > 260]
display(wrong_ap_hi.head())
wrong_ap_lo = df_train[df_train['ap_lo'] > 150]
display(wrong_ap_lo.head())
nul_ap_hi = df_train[df_train['ap_hi'] == 0]
display(nul_ap_hi.head())
nul_ap_lo = df_train[df_train['ap_lo'] == 0]
display(nul_ap_lo.head())

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
2654,41,0,160.0,60.0,902,60,1,1,0,0,1,0
2845,62,1,167.0,59.0,906,0,1,1,0,0,1,0
6822,39,0,168.0,63.0,909,60,2,1,0,0,1,0
11089,57,0,175.0,80.0,11500,90,1,1,0,0,1,1
12710,51,0,164.0,75.0,1420,80,2,1,0,0,1,1


,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
314,47,1,183.0,98.0,160,1100,1,2,1,0,1,1
334,60,1,157.0,60.0,160,1000,2,1,0,0,0,1
357,49,0,150.0,83.0,140,800,1,1,0,0,1,1
458,64,0,176.0,63.0,160,1000,2,2,0,0,0,1
482,51,0,154.0,81.0,140,1000,2,1,0,0,1,1


,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,


,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
2845,62,1,167.0,59.0,906,0,1,1,0,0,1,0
19258,40,1,150.0,60.0,130,0,1,1,0,0,0,0
23512,56,0,156.0,50.0,138,0,3,1,0,0,1,1
24837,49,0,164.0,78.0,138,0,1,1,0,0,1,0
32749,57,1,166.0,68.0,149,0,1,1,0,0,1,1


In [25]:
def ap_hi(row):
      
    if row['ap_hi'] > 260 and row['ap_hi'] <= 2000:
        ap_hi = int(row['ap_hi'] / 10)
        row['ap_hi'] = ap_hi
    elif row['ap_hi'] > 2000:
        ap_hi = int(row['ap_hi'] / 100)
        row['ap_hi'] = ap_hi
        
    return row

df_train = df_train.apply(ap_hi, axis=1)

In [26]:
display(df_train.loc[wrong_ap_hi.index].head())

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
2654,41.0,0.0,160.0,60.0,90.0,60.0,1.0,1.0,0.0,0.0,1.0,0.0
2845,62.0,1.0,167.0,59.0,90.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0
6822,39.0,0.0,168.0,63.0,90.0,60.0,2.0,1.0,0.0,0.0,1.0,0.0
11089,57.0,0.0,175.0,80.0,115.0,90.0,1.0,1.0,0.0,0.0,1.0,1.0
12710,51.0,0.0,164.0,75.0,142.0,80.0,2.0,1.0,0.0,0.0,1.0,1.0


In [27]:
df_train.loc[df_train['ap_lo'] > 150, 'ap_lo'] /= 10
display(df_train.loc[wrong_ap_lo.index].head())

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
314,47.0,1.0,183.0,98.0,160.0,110.0,1.0,2.0,1.0,0.0,1.0,1.0
334,60.0,1.0,157.0,60.0,160.0,100.0,2.0,1.0,0.0,0.0,0.0,1.0
357,49.0,0.0,150.0,83.0,140.0,80.0,1.0,1.0,0.0,0.0,1.0,1.0
458,64.0,0.0,176.0,63.0,160.0,100.0,2.0,2.0,0.0,0.0,0.0,1.0
482,51.0,0.0,154.0,81.0,140.0,100.0,2.0,1.0,0.0,0.0,1.0,1.0


In [28]:
df_train.loc[nul_ap_lo.index, 'ap_lo'] = None
df_train['ap_lo'] = df_train['ap_lo'].fillna(df_train['ap_lo'].median())
display(df_train.loc[nul_ap_lo.index].head())

,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio
id,,,,,,,,,,,,
2845,62.0,1.0,167.0,59.0,90.0,80.0,1.0,1.0,0.0,0.0,1.0,0.0
19258,40.0,1.0,150.0,60.0,130.0,80.0,1.0,1.0,0.0,0.0,0.0,0.0
23512,56.0,0.0,156.0,50.0,138.0,80.0,3.0,1.0,0.0,0.0,1.0,1.0
24837,49.0,0.0,164.0,78.0,138.0,80.0,1.0,1.0,0.0,0.0,1.0,0.0
32749,57.0,1.0,166.0,68.0,149.0,80.0,1.0,1.0,0.0,0.0,1.0,1.0


In [29]:
import seaborn as sns
sns.histplot(data=df_train)

<AxesSubplot:ylabel='Count'>

In [30]:
df_train['bmi'] = df_train['weight']/(df_train['height']/100)**2
display(df_train.head())
df_test['bmi'] = df_test['weight']/(df_test['height']/100)**2
display(df_test.head())


,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,bmi
id,,,,,,,,,,,,,
0,50.0,1.0,168.0,62.0,110.0,80.0,1.0,1.0,0.0,0.0,1.0,0.0,21.967120
1,55.0,0.0,156.0,85.0,140.0,90.0,3.0,1.0,0.0,0.0,1.0,1.0,34.927679
2,51.0,0.0,165.0,64.0,130.0,70.0,3.0,1.0,0.0,0.0,0.0,1.0,23.507805
3,48.0,1.0,169.0,82.0,150.0,100.0,1.0,1.0,0.0,0.0,1.0,1.0,28.710479
4,47.0,0.0,156.0,56.0,100.0,60.0,1.0,1.0,0.0,0.0,0.0,0.0,23.011177


,age,gender,height,weight,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,bmi
id,,,,,,,,,,,,
5,51,0,154,85.0,130,80,1,1,0,0,1,35.840783
6,52,1,170,69.0,130,90,1,1,0,0,1,23.875433
7,55,0,160,70.0,120,75,1,1,0,0,0,27.343750
10,49,1,185,94.0,130,90,1,1,0,0,1,27.465303
11,44,1,170,76.0,120,80,1,1,0,0,1,26.297578


In [31]:
df_train.drop(columns=['height', 'weight'], inplace=True)
display(df_train.head())
df_test.drop(columns=['height', 'weight'], inplace=True)
display(df_test.head())

,age,gender,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,cardio,bmi
id,,,,,,,,,,,
0,50.0,1.0,110.0,80.0,1.0,1.0,0.0,0.0,1.0,0.0,21.967120
1,55.0,0.0,140.0,90.0,3.0,1.0,0.0,0.0,1.0,1.0,34.927679
2,51.0,0.0,130.0,70.0,3.0,1.0,0.0,0.0,0.0,1.0,23.507805
3,48.0,1.0,150.0,100.0,1.0,1.0,0.0,0.0,1.0,1.0,28.710479
4,47.0,0.0,100.0,60.0,1.0,1.0,0.0,0.0,0.0,0.0,23.011177


,age,gender,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,bmi
id,,,,,,,,,,
5,51,0,130,80,1,1,0,0,1,35.840783
6,52,1,130,90,1,1,0,0,1,23.875433
7,55,0,120,75,1,1,0,0,0,27.343750
10,49,1,130,90,1,1,0,0,1,27.465303
11,44,1,120,80,1,1,0,0,1,26.297578


In [32]:
df_train.shape

(69976, 11)

In [33]:
df_test.shape

(30000, 10)

In [34]:
print(df_train['cardio'].value_counts())

0.0    35004
1.0    34972
Name: cardio, dtype: int64


In [35]:
categorical_features = ['gender', 'smoke', 'alco', 'active']
int_features = ['age','ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'cardio']
df_train[categorical_features] = df_train[categorical_features].astype('category')
df_train[int_features] = df_train[int_features].astype('int64')
df_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 69976 entries, 0 to 99999
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype   
---  ------       --------------  -----   
 0   age          69976 non-null  int64   
 1   gender       69976 non-null  category
 2   ap_hi        69976 non-null  int64   
 3   ap_lo        69976 non-null  int64   
 4   cholesterol  69976 non-null  int64   
 5   gluc         69976 non-null  int64   
 6   smoke        69976 non-null  category
 7   alco         69976 non-null  category
 8   active       69976 non-null  category
 9   cardio       69976 non-null  int64   
 10  bmi          69976 non-null  float64 
dtypes: category(4), float64(1), int64(6)
memory usage: 6.6 MB


In [36]:
df_train_numeric = df_train.select_dtypes(include=[np.number])
df_train_numeric_cols = df_train_numeric.columns.values.tolist()
print('Столбцы с количественными признаками тренировочная выборка: \n', df_train_numeric_cols)

Столбцы с количественными признаками тренировочная выборка: 
 ['age', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'cardio', 'bmi']


In [37]:
features_train = df_train.drop(['cardio'], axis=1)
target_train = df_train['cardio']

In [38]:
sc = MinMaxScaler()
features_train_sc = features_train.copy()
features_train_sc[['age', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'bmi']] = sc.fit_transform(features_train[['age', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'bmi']])
#features_train = pd.DataFrame(df_train, index=df_train.index, columns=df_train.columns)
display(features_train_sc.head())

features_test_sc = df_test.copy()
features_test_sc[['age', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'bmi']] = sc.transform(features_test_sc[['age', 'ap_hi', 'ap_lo', 'cholesterol', 'gluc', 'bmi']])
#features_test = pd.DataFrame(features_test_sc, index=fd_test.index, columns=df_test.columns)

features_test_sc.head()


,age,gender,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,bmi
id,,,,,,,,,,
0,0.600000,1.0,0.456067,0.071884,0.0,0.0,0.0,0.0,1.0,0.079256
1,0.742857,0.0,0.581590,0.080983,1.0,0.0,0.0,0.0,1.0,0.170640
2,0.628571,0.0,0.539749,0.062784,1.0,0.0,0.0,0.0,0.0,0.090119
3,0.542857,1.0,0.623431,0.090082,0.0,0.0,0.0,0.0,1.0,0.126803
4,0.514286,0.0,0.414226,0.053685,0.0,0.0,0.0,0.0,0.0,0.086617


,age,gender,ap_hi,ap_lo,cholesterol,gluc,smoke,alco,active,bmi
id,,,,,,,,,,
5,0.628571,0,0.539749,0.071884,0.0,0.0,0,0,1,0.177078
6,0.657143,1,0.539749,0.080983,0.0,0.0,0,0,1,0.092711
7,0.742857,0,0.497908,0.067334,0.0,0.0,0,0,0,0.117166
10,0.571429,1,0.539749,0.080983,0.0,0.0,0,0,1,0.118023
11,0.428571,1,0.497908,0.071884,0.0,0.0,0,0,1,0.109790


In [39]:
# %%time

# lg_params = {
#     'penalty':['l1','l2'],
#     'C':[0.001, 0.01, 0.1, 1, 10, 100, 1000],
#     'solver':['saga'],
#     'class_weight':['balanced']}

# lg_gs = GridSearchCV(LogisticRegression(random_state=12345),
#                  lg_params,
#                  cv = 5,
#                  scoring = 'roc_auc')
# lg = lg_gs.fit(features_train_sc, target_train)

In [40]:
# print('Оптимальные параметры Логистическая регрессия: ', lg.best_params_)
# print('Лучший показатель AUC_ROC Логистическая регрессия:', lg.best_score_)

In [41]:
# %%time
# dt_params = {
#     'max_depth': [2, 4, 8, 16],
#     'min_samples_split': range(5,55,10), 
#     'min_samples_leaf': [2, 3, 5, 10],
#     'class_weight':['balanced']
# }

# dt_gs_auc = GridSearchCV(DecisionTreeClassifier(random_state=12345),
#                          dt_params, n_jobs=-1, cv = 5, scoring = 'roc_auc')
# dt_auc = dt_gs_auc.fit(features_train_sc, target_train)

In [42]:
# print('Оптимальные параметры Решающее дерево: ', dt_auc.best_params_)
# print('Лучший показатель AUC_ROC Решающее дерево:', dt_auc.best_score_)

In [43]:
# %%time
# rf_params = {
#     'criterion':['gini','entropy'],
#     'n_estimators': [2, 5, 10, 30, 50],
#     'max_depth': [2, 4, 8, 16],
#     'min_samples_leaf': [2,3,5,10],
#     'class_weight':['balanced']
# }

# rf_gs = GridSearchCV(RandomForestClassifier(random_state=12345),
#                      rf_params, cv = 5, scoring = 'roc_auc',
#                      verbose=2)
# rf_auc = rf_gs.fit(features_train_sc, target_train)

In [44]:
# %%time
# rf_params = {
#     'criterion':['gini','entropy'],
#     'n_estimators': [60,80,100],
#     'max_depth': [20, 25],
#     'min_samples_leaf': [15,20],
#     'class_weight':['balanced',None]
# }

# rf_gs = GridSearchCV(RandomForestClassifier(random_state=12345),
#                      rf_params, cv = 5, scoring = 'roc_auc',
#                      verbose=2)
# rf_auc = rf_gs.fit(features_train_sc, target_train)

In [45]:
# print('Оптимальные параметры Случайный лес: ', rf_auc.best_params_)
# print('Лучший показатель AUC_ROC Случайный лес:', rf_auc.best_score_)

In [46]:
# predictive_model_scores = pd.DataFrame(data=[('Logistic Regression', lg.best_score_), 
#                                              ('Decision Tree', dt_auc.best_score_), 
#                                              ('Random Forest', rf_auc.best_score_)], 
#                                        columns=['model', 'AUC_ROC'])
# predictive_model_scores

In [47]:
#from sklearn.neighbors                  import KNeighborsClassifier
#from sklearn.svm                        import SVC
#from sklearn.gaussian_process           import GaussianProcessClassifier
#from sklearn.gaussian_process.kernels   import RBF
#from sklearn.ensemble                   import AdaBoostClassifier, ExtraTreesClassifier
#from sklearn.neural_network             import MLPClassifier
#from sklearn.naive_bayes                import GaussianNB
#from sklearn.discriminant_analysis      import QuadraticDiscriminantAnalysis

#classifiers = {
    #"KNN": KNeighborsClassifier(3),
    #"SVC_lin": SVC(kernel="linear", C=0.025),
    #"SVC": SVC(),
    #"Gaussian": GaussianProcessClassifier(1.0 * RBF(1.0)),
    #"ExtraTrees": ExtraTreesClassifier(),
    #"MLPC": MLPClassifier(alpha=1, max_iter=1000),
    #"AdaBoost": AdaBoostClassifier(),
    #"GradientBoosting": GradientBoostingClassifier(),
    #"GaussianNB": GaussianNB(),
    #"Quadratic": QuadraticDiscriminantAnalysis()
    
#}

In [48]:
#scores = {}
#for name, classifier in classifiers.items():
    #score = cross_val_score(classifier, features_train_sc, target_train, cv=10, scoring='roc_auc').mean()
    #scores[name] = score
    
#pd.DataFrame.from_dict(scores, orient="index", columns=['score']).sort_values(by="score", ascending=False)

In [49]:
cls = RandomForestClassifier(random_state=12345, max_depth=20, n_estimators=100, min_samples_leaf=20, criterion='gini', class_weight=None)
cls.fit(features_train_sc, target_train)

RandomForestClassifier(max_depth=20, min_samples_leaf=20, random_state=12345)

In [50]:
from sklearn.model_selection import RandomizedSearchCV
grb = GradientBoostingClassifier(random_state=12345)
param_distributions = {
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'n_estimators': [50, 100, 200, 300],
    'max_depth': [3, 5, 7, 9],
    'min_samples_split': [2, 5, 10, 15],
    'min_samples_leaf': [1, 2, 4, 6],
    'max_features': [None, 'auto', 'sqrt', 'log2']
}
rs_grb = RandomizedSearchCV(grb, 
                               param_distributions=param_distributions, 
                               cv=5, 
                               n_iter=20, 
                               n_jobs=-1, 
                               random_state=12345,
                               verbose=2,
                               scoring='roc_auc')

rs_grb.fit(features_train_sc, target_train)

Fitting 5 folds for each of 20 candidates, totalling 100 fits


RandomizedSearchCV(cv=5,
                   estimator=GradientBoostingClassifier(random_state=12345),
                   n_iter=20, n_jobs=-1,
                   param_distributions={'learning_rate': [0.01, 0.05, 0.1, 0.2],
                                        'max_depth': [3, 5, 7, 9],
                                        'max_features': [None, 'auto', 'sqrt',
                                                         'log2'],
                                        'min_samples_leaf': [1, 2, 4, 6],
                                        'min_samples_split': [2, 5, 10, 15],
                                        'n_estimators': [50, 100, 200, 300]},
                   random_state=12345, scoring='roc_auc', verbose=2)

In [51]:
predictions = rs_grb.predict(features_test_sc)

to_submit = pd.DataFrame(index=features_test_sc.index)
to_submit['cardio'] = predictions
to_submit['cardio'] = to_submit['cardio'].astype('float16')
to_submit.to_csv('submission.csv')
print(to_submit.shape)

(30000, 1)


In [52]:
from joblib import dump
dump(cls, 'heart.model')

['heart.model']

In [53]:
to_submit.head()

,cardio
id,
5,0.0
6,1.0
7,0.0
10,1.0
11,0.0
